In [7]:
import os
import numpy as np
import codecs
import random

In [2]:
def get_sentences_from_src_tgt(file_name):
    src_words=[]
    tgt_words=[]
    with codecs.open(file_name, encoding='utf-8') as f:
#         sentences_words=[]
        for line in f.readlines():
            line_array=line.strip().split('\t')
#             print(line_array[0])
            if len(line_array)>1:
                src_words.append(line_array[0])
                tgt_words.append(line_array[1])
        f.close()
    return src_words,tgt_words

def save_to_txt(sentences_array,file_output_name):
    if os.path.isfile(file_output_name):
        os.remove(file_output_name)
    with codecs.open(file_output_name, "a",encoding='utf-8') as a_file:
        for elem in sentences_array:
            a_file.write(elem)
            a_file.write("\n")
    a_file.close()

In [ ]:
def get_sentences_from_all(file_name):
    src_words=[]
    tgt_words=[]
    all_src_tgt=[]
    with codecs.open(file_name, encoding='utf-8') as f:
#         sentences_words=[]
        for line in f.readlines():
            line_array=line.strip().split('\t')
            all_src_tgt.append(line_array)
#             print(line_array[0])
            if len(line_array)>1:
                src_words.append(line_array[0])
                tgt_words.append(line_array[1])
        f.close()
    return src_words,tgt_words,all_src_tgt

In [ ]:
%%time
src_words,tgt_words,all_src_tgt=get_sentences_from_all("C:/Users/Hanane/Downloads/data_Wiki_NER_filtered.sent-ner")

In [11]:
%%time
src_words,tgt_words=get_sentences_from_src_tgt("C:/Users/Hanane/Downloads/data_Wiki_NER_filtered.sent-ner")

Wall time: 2min 46s


In [13]:
whole_dataset

[('" " ( Editions législatives , novembre 2017', 'O O O O O O B-DATE I-DATE'),
 ('" " ( Editions législatives , novembre', 'O O O O O O B-DATE'),
 ('" " , avec Robyn', 'O O O O B-PEOPLE'),
 ('" " ( Editions législatives , novembre 2017 )',
  'O O O O O O B-DATE I-DATE O'),
 ('" " , avec Robyn , a', 'O O O O B-PEOPLE O O'),
 ('" " , jouée au Théâtre des Déchargeurs à',
  'O O O O O B-MISC I-MISC I-MISC O'),
 ('" " , le 14 février 2011. Khemis El',
  'O O O O B-DATE I-DATE O B-LOC_CITY I-LOC_CITY'),
 ('" " , avec Robyn , a été', 'O O O O B-PEOPLE O O O'),
 ('" " , avec Robyn ,', 'O O O O B-PEOPLE O'),
 ('" " , avec Robyn , a été publié', 'O O O O B-PEOPLE O O O O')]

In [16]:
from time import time

work with all_src_tgt instead of zipping the whole data set ==> and try to get words and tags by slicing the arrays instead of comprehensive lists

In [17]:
print("in zip")
t0=time()
whole_dataset=list(zip(src_words,tgt_words))
print("zip %0.3fs." % (time() - t0)," %0.1fs." % (time() - t0)/60)
    
t0=time()
random.shuffle(whole_dataset)
print("shuffle %0.3fs." % (time() - t0)," %0.1fs." % (time() - t0)/60)

t0=time()
train_size=int(0.8*len(whole_dataset))
valid_size=int(0.9*len(whole_dataset))
train_set=whole_dataset[:train_size]
valid_set=whole_dataset[train_size:valid_size]
test_set=whole_dataset[valid_size:]
print("build train-valid-test %0.3fs." % (time() - t0)," %0.1fs." % (time() - t0)/60)


t0=time()
valid_words=[elem[0] for elem in valid_set]
valid_tags=[elem[1] for elem in valid_set]
print("build valid %0.3fs." % (time() - t0),," %0.1fs." % (time() - t0)/60)

t0=time()
test_words=[elem[0] for elem in test_set]
test_tags=[elem[1] for elem in test_set]
print("build test %0.2fs." % (time() - t0)," %0.1fs." % (time() - t0)/60)

t0=time()
train_words=[elem[0] for elem in train_set]
train_tags=[elem[1] for elem in train_set]
print("build train %0.3fs." % (time() - t0)," %0.1fs." % (time() - t0)/60)

in zip
zip 319.613s.
shuffle 71.278s.
build train-valid-test 1.522s.
build valid 509.012s.
build test 585.912s.
build train 3141.623s.


In [24]:
valid_tags[:10]

['O O O O O O B-LOC_CITY',
 'B-DATE I-DATE I-DATE O B-LOC_CITY',
 'O O O O B-MISC I-MISC O',
 'B-DATE O O O O O',
 'I-PEOPLE O B-PEOPLE I-PEOPLE',
 'O O B-LOC_COUNTRY O',
 'O O O O O O B-DATE O',
 'O O O O O O O O B-MISC',
 'O O O O O O B-LOC_CITY O',
 'O O B-PEOPLE I-PEOPLE O O O O']

In [25]:
%%time
save_to_txt(train_words,"../dataset/train_valid/wiki_NER_semi_auto/wiki_train_words_bitext_large_2.txt")
save_to_txt(train_tags,"../dataset/train_valid/wiki_NER_semi_auto/wiki_train_tags_bitext_large_2.txt")

In [26]:
%%time
save_to_txt(valid_words,"../dataset/train_valid/wiki_NER_semi_auto/wiki_valid_words_bitext_large_2.txt")
save_to_txt(valid_tags,"../dataset/train_valid/wiki_NER_semi_auto/wiki_valid_tags_bitext_large_2.txt")

Wall time: 3min 39s


In [27]:
%%time
save_to_txt(test_words,"../dataset/train_valid/wiki_NER_semi_auto/wiki_test_words_bitext_large_2.txt")
save_to_txt(test_tags,"../dataset/train_valid/wiki_NER_semi_auto/wiki_test_tags_bitext_large_2.txt")

Wall time: 2min 49s


### Generate vocabularies ==> word, tag and tkt

##### Word vocab

In [28]:
%%time
#254294 rows
path="../dataset/train_valid/wiki_NER_semi_auto/"
!onmt-build-vocab --size 1000000 --save_vocab ../dataset/train_valid/wiki_NER_semi_auto/wiki-src-train-vocab.txt ../dataset/train_valid/wiki_NER_semi_auto/wiki_train_words_bitext_large_2.txt

2020-08-04 14:40:27.258395: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-08-04 14:40:27.259954: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-08-04 14:40:40.039696: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2020-08-04 14:40:40.040643: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2020-08-04 14:40:40.055900: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-79FP3JFQ
2020-08-04 14:40:40.057207: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-79FP3JFQ
2020-08-04 14:40:40.065286: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instruc

##### Target vocab

In [29]:
%%time
!onmt-build-vocab --size 1000 --save_vocab ../dataset/train_valid/wiki_NER_semi_auto/wiki-tgt-train-vocab.txt ../dataset/train_valid/wiki_NER_semi_auto/wiki_train_tags_bitext_large_2.txt

Wall time: 8min 40s


2020-08-04 15:02:09.899109: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-08-04 15:02:09.899935: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-08-04 15:02:17.243072: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2020-08-04 15:02:17.243944: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2020-08-04 15:02:17.257385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-79FP3JFQ
2020-08-04 15:02:17.258589: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-79FP3JFQ
2020-08-04 15:02:17.259719: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instruc

##### Tokenize Characters

In [30]:
%%time
!onmt-build-vocab --tokenizer_config ../config/char_tokenization.yml --size 1000000 --save_vocab ../dataset/train_valid/wiki_NER_semi_auto/wiki-src-train-tkt-vocab.txt ../dataset/train_valid/wiki_NER_semi_auto/wiki_train_words_bitext_large_2.txt

Wall time: 19min 49s


2020-08-04 15:11:37.211163: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-08-04 15:11:37.212027: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2020-08-04 15:11:44.211053: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2020-08-04 15:11:44.211852: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2020-08-04 15:11:44.224522: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-79FP3JFQ
2020-08-04 15:11:44.225639: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-79FP3JFQ
2020-08-04 15:11:44.226712: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instruc